In [1]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import numpy as np
from sklearn.linear_model import Lasso
from scipy.io import loadmat
from sklearn.metrics import mean_squared_error
from scipy.integrate import solve_ivp

import pysindy as ps

import os

from get_model import getmodel
from compare_center_not_show import cmp_center

import math  
import sys  
sys.path.append('C:/Users/j/hotai/myfolder')

from my_function import miscore, siscore
# Ignore matplotlib deprecation warnings
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

# Seed the random number generators for reproducibility   sr_C1_20  sr_C4_12
np.random.seed(100)

integrator_keywords = {}
integrator_keywords['rtol'] = 1e-12
integrator_keywords['method'] = 'LSODA'
integrator_keywords['atol'] = 1e-12

addp=5
startp=90
endp=120
dp=endp-startp
p_tot=addp*(dp-1)+1
file_tot=100
n=25
xstart=0
xend=25

In [2]:
def read_data(file_path,file_name,n,file_start=0,file_end=0,xstart=0,xend=0):
    x=np.zeros((121,n,n))
    y=np.zeros((121,n,n))
    vx=np.zeros((121,n,n))
    vy=np.zeros((121,n,n))
    #file_tot=file_end-file_start
    for i in range(file_start,file_end):
        xp=[]
        yp=[]
        vxp=[]
        vyp=[]
        isvalid=[]
        xtmp=[]
        ytmp=[]
        vxtmp=[]
        vytmp=[]
        isvalidtmp=[]
        
        for root,dirs,files in os.walk(file_path):
            #print(root)
            
            for dir in dirs:
                #print(os.path.join(root,file))
                
    
                    with open(os.path.join(root,dir,file_name[i]), 'r') as f:  
                        lines = f.readlines()
                        for line in lines[3:]: 
                            num=line.split()
                            xtmp.append(float(num[0]))
                            ytmp.append(float(num[1]))
                            vxtmp.append(float(num[2]))
                            vytmp.append(float(num[3]))
                            isvalidtmp.append(float(num[4]))
        
                    xp.append(np.array(xtmp).reshape(25,25))
                    yp.append(np.array(ytmp).reshape(25,25))
                    vxp.append(np.array(vxtmp).reshape(25,25))
                    vyp.append(np.array(vytmp).reshape(25,25))
                    isvalid.append(np.array(isvalidtmp).reshape(25,25))
                    xtmp=[]
                    ytmp=[]
                    vxtmp=[]
                    vytmp=[]
                    isvalidtmp=[]
        #
        xp=np.array(xp)
        yp=np.array(yp)
        vxp=np.array(vxp)
        vyp=np.array(vyp)
        isvalid=np.array(isvalid)
        x=x+xp/file_tot
        y=y+yp/file_tot
        vx=vx+vxp/file_tot
        vy=vy+vyp/file_tot
    x=x.T
    x=x[xstart:xend,xstart:xend]
    y=y.T
    y=y[xstart:xend,xstart:xend]
    vx=vx.T
    vx=vx[xstart:xend,xstart:xend]
    vy=vy.T
    vy=vy[xstart:xend,xstart:xend]
    isvalid=isvalid.T
    isvalid=isvalid[xstart:xend,xstart:xend]
    return x,y,vx,vy

In [3]:
file_path = os.path.join('D:\\', 'hotaidata','sr_C4_12')
file_name=['B00001.dat','B00002.dat','B00003.dat','B00004.dat','B00005.dat','B00006.dat','B00007.dat','B00008.dat',
           'B00009.dat','B00010.dat','B00011.dat','B00012.dat','B00013.dat','B00014.dat','B00015.dat','B00016.dat','B00017.dat',
           'B00018.dat','B00019.dat','B00020.dat','B00021.dat','B00022.dat','B00023.dat','B00024.dat','B00025.dat','B00026.dat',
           'B00027.dat','B00028.dat','B00029.dat','B00030.dat','B00031.dat','B00032.dat','B00033.dat','B00034.dat','B00035.dat',
           'B00036.dat','B00037.dat','B00038.dat','B00039.dat','B00040.dat','B00041.dat','B00042.dat','B00043.dat','B00044.dat',
           'B00045.dat','B00046.dat','B00047.dat','B00048.dat','B00049.dat','B00050.dat','B00051.dat','B00052.dat','B00053.dat',
           'B00054.dat','B00055.dat','B00056.dat','B00057.dat','B00058.dat','B00059.dat','B00060.dat','B00061.dat','B00062.dat',
           'B00063.dat','B00064.dat','B00065.dat','B00066.dat','B00067.dat','B00068.dat','B00069.dat','B00070.dat','B00071.dat',
           'B00072.dat','B00073.dat','B00074.dat','B00075.dat','B00076.dat','B00077.dat','B00078.dat','B00079.dat','B00080.dat',
           'B00081.dat','B00082.dat','B00083.dat','B00084.dat','B00085.dat','B00086.dat','B00087.dat','B00088.dat','B00089.dat',
           'B00090.dat','B00091.dat','B00092.dat','B00093.dat','B00094.dat','B00095.dat','B00096.dat','B00097.dat','B00098.dat',
           'B00099.dat','B00100.dat']
x=np.zeros((n,n,121))
y=np.zeros((n,n,121))
vx=np.zeros((n,n,121))
vy=np.zeros((n,n,121))

#print(x[0,:,:])

In [4]:
#u is not pre adding, v add a median

x,y,vx,vy=read_data(file_path,file_name,25,0,50,xstart,xend)

spatial_grid = np.asarray([x[:,:,0].T,y[:,:,0].T]).T

n=xend-xstart

u=np.zeros((n,n,121, 3))
u[:, :, :, 0] = vx
u[:, :, :, 1] = vy

t=np.linspace(0,60/800,121)

t_in=np.zeros((n,n,121))
for i in range (len(t)):
    t_in[:,:,i]=t[i]

u[:, :, :, 2] = t_in

dt=t[1]-t[0]

t_adding=np.linspace(t[0],t[-1],addp*(len(t)-1)+1)
print(t_adding.shape)
dt_adding=t_adding[1]-t_adding[0]

u_train=np.zeros((n,n,len(t_adding),3))

for i in range(len(t)-1):
    for j in range(addp):
        u_train[:,:,addp*i+j]=(u[:,:,i]*(addp-j)+u[:,:,i+1]*j)/addp
u_train[:,:,-1]=u[:,:,-1]


(601,)


In [5]:
# import test data
x_test,y_test,vx_test,vy_test=read_data(file_path,file_name,25,51,100,xstart,xend)
ut=np.zeros((n,n,121, 3))

ut[:, :, :, 0] = vx_test
ut[:, :, :, 1] = vy_test
ut[:, :, :, 2] = t_in

u_test=np.zeros((n,n,len(t_adding), 3))

for i in range(len(t)-1):
    for j in range(addp):
        u_test[:,:,addp*i+j]=(ut[:,:,i]*(addp-j)+ut[:,:,i+1]*j)/addp
u_test[:,:,-1]=ut[:,:,-1]

si_t=np.zeros(p_tot)
mi_t=np.zeros(p_tot)
si_t_per=np.zeros(p_tot)
mi_t_per=np.zeros(p_tot)

In [6]:
left_bond=70
test_range=20
good_time=np.zeros(test_range)

v_train=np.zeros((n,n,p_tot,3))
t_train=np.zeros(p_tot)
v_test=np.zeros((n,n,p_tot,3))



for k in range(test_range):
    print('round:',k)
    startp=left_bond+k
    endp=left_bond+k+dp
    v_train=u_train[:,:,startp*addp:(endp-1)*addp+1]   
    t_train=v_train[0,0,:,2]
    
    dt_train=v_train[0,0,1,2]-v_train[0,0,0,2]
    
    
    v_test=u_test[:,:,startp*addp:(endp-1)*addp+1]
    
    print(v_train.shape)
    print(t_train.shape)
    
    
    '''
    np.save('v_train.npy',v_train)
    np.savetxt('v_train_dot.txt',v_train_dot)
    np.savetxt('t_train.txt',t_train)
    '''
    model=getmodel(v_train,spatial_grid, dt_train)
    
    x_dot_test_predicted = model.predict(v_test)
    x_dot_test_predicted=x_dot_test_predicted.reshape(n,n,p_tot,3)
    tlen=p_tot
    dot_tmp=np.array(x_dot_test_predicted)
    pltpredict=np.zeros((n,n,tlen,2))
    #print(dot_tmp[10,10])

    
    pltpredict[:,:,0]=v_train[:,:,0,:2]
    for i in range(tlen-1):
        pltpredict[:,:,i+1]=pltpredict[:,:,i]+dot_tmp[:,:,i,:2]*dt_train

    map_num=p_tot
    center=np.zeros((p_tot,6))
    
    strength=np.zeros((map_num,2))
    bias_center=np.zeros(map_num)
    
    for i in range(p_tot):
        j=i
        mi_t[i],mi_t_per[i]=miscore(pltpredict[:,:,j,],v_test[:,:,j,:])
        si_t[i],si_t_per[i]=siscore(pltpredict[:,:,j,],v_test[:,:,j,:])
        center[j],bias_center[j],strength[j] = cmp_center(v_test[:,::-1,j,0].T,v_test[:,::-1,j,1].T,pltpredict[:,::-1,j,0].T,pltpredict[:,::-1,j,1].T,n)
        if mi_t_per[i]>0.8 and si_t_per[i]>0.8:
            good_time[k]=good_time[k]+1

        '''
        if i%(addp)==0:
            plt.figure(figsize=(16, 4))
            plt.subplot(1, 3, 1)
            plt.title('MI percetage '+str(mi_t_per[i]))
            plt.quiver(spatial_grid[:,:,0],spatial_grid[:,:,1],v_test[:,:,j,0],v_test[:,:,j,1],color='r',label='real')
            plt.quiver(spatial_grid[:,:,0],spatial_grid[:,:,1],pltpredict[:,:,j,0],pltpredict[:,:,j,1],color='b',label='predict')
            plt.legend()
            plt.subplot(1, 3, 2)
            plt.title('SI percetage '+str(si_t_per[i]))
            plt.quiver(spatial_grid[:,:,0],spatial_grid[:,:,1],v_test[:,:,j,0],v_test[:,:,j,1],color='r',label='real')
            #plt.quiver(spatial_grid[:,:,1],spatial_grid[:,:,0],pltpredict[:,:,i,0],pltpredict[:,:,i,1],color='b')
            plt.legend()
            plt.subplot(1, 3, 3)
            #plt.quiver(spatial_grid[:,:,1],spatial_grid[:,:,0],u_dot_train[:,:,i+1,0],u_dot_train[:,:,i+1,1],color='r')
            plt.quiver(spatial_grid[:,:,0],spatial_grid[:,:,1],pltpredict[:,:,j,0],pltpredict[:,:,j,1],color='b',label='predict')
            plt.legend()
            plt.show()
        '''
    
    plt.figure(figsize=(12, 4))
    plt.title('MI/SI~t(SINDy)')
    plt.ylim(0,1)
    plt.xlabel('t')
    plt.ylabel('mi/si')
    plt.plot(t_train,mi_t,'r',label='MI(average)')
    plt.plot(t_train,si_t,'b',label='SI(average)')
    plt.plot(t_train,mi_t_per,'r--',label='MI(percentage)')
    plt.plot(t_train,si_t_per,'b--',label='SI(percentage)')
    plt.legend()
    plt.ylim(0.0, 1)
    plt.plot
    plt.show()

    plt.figure(figsize=(12, 4))
    plt.title('bias_center')
    #plt.ylim(0,1)
    plt.xlabel('t')
    plt.ylabel('bias')
    plt.plot(t_train,bias_center,'r',label='dist')
    
    plt.legend()
    #plt.ylim(0.5, 1)
    plt.plot
    plt.show()
    
    plt.figure(figsize=(12, 4))
    plt.title('strength')
    
    plt.xlabel('t')
    plt.ylabel('strength')
    plt.plot(t_train,strength[:,0],'r',label='real')
    plt.plot(t_train,strength[:,1],'b',label='predict')
    
    plt.legend()
    
    plt.show()
    
    good_time[k]=good_time[k]/addp



round: 0
(25, 25, 146, 3)
(146,)
SINDy
(Vx)' = -107.307 1 + -123.461 Vx + 159.566 Vy + 3342.618 t + 31.447 VxVx + -57.495 VyVy + -23310.048 tt + 78.182 VxVy + 2265.694 Vxt + -2621.571 Vyt + -9.438 VxVxVy + -200.865 VxVxt + 1092.320 VyVyt + -1428.170 VxVyt + 2093.200 Vx_2 + -2296.517 Vy_2 + 115.699 Vx_1 + 1220.512 Vy_1 + -533.508 VxVx_2 + -624.450 VxVy_2 + -82032.986 Vxt_2 + -93.582 VyVx_2 + 515.344 VyVy_2 + 804687.465 Vyt_2 + 191.755 tVx_2 + 8643.118 tVy_2 + 10952.856 tt_2 + 26.367 VxVxVx_2 + 1176.256 VxVxVy_2 + -9680.534 VxVxt_2 + -2393.147 VyVyVx_2 + 202.222 VyVyVy_2 + -97.473 VyVyt_2 + 87438.914 ttVx_2 + -370.072 ttVy_2 + -36.530 ttt_2 + -820304.704 VxVyVx_2 + -540.591 VxVyVy_2 + -4430.893 VxVyt_2 + 1607.874 VxtVx_2 + -27.341 VxtVy_2 + 6882.791 Vxtt_2 + 651.470 VytVx_2 + 9745.817 VytVy_2 + -599.903 VxVx_1 + 418.762 VxVy_1 + -13021.278 Vxt_1 + 27.966 VyVx_1 + 318.331 VyVy_1 + 198022.255 Vyt_1 + 849.437 tVx_1 + 10568.653 tVy_1 + -6184.201 tt_1 + -5.235 VxVxVx_1 + -177.242 VxVxVy_1 + -

UnboundLocalError: local variable 'adj_center_x' referenced before assignment

In [ ]:
print('fit_time:',good_time)
plt.figure(figsize=(12, 4))
plt.plot(range(test_range),good_time,'r')
plt.show